In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import vtk
import math
from vtk.util import numpy_support

from params import *



In [2]:
def vti_reader(file_path):
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(file_path)
    reader.Update()
    data = reader.GetOutput()
    
    # vtk res
    dataset = pd.DataFrame(np.array([numpy_support.vtk_to_numpy(data.GetPointData().GetArray(i)) for i in range(data.GetPointData().GetNumberOfArrays())])).transpose()
    
    dataset.columns=[data.GetPointData().GetArrayName(i) for i in range(data.GetPointData().GetNumberOfArrays())]
    
    # vtk coord
    dimensions = data.GetDimensions()
    spacing=data.GetSpacing()
    origin=data.GetOrigin()
    
    coordinates=[]
    
    for k in range(dimensions[2]):
        for j in range(dimensions[1]):
            for i in range(dimensions[0]):
                x = origin[0] + i * spacing[0]
                y = origin[1] + j * spacing[1]
                z = origin[2] + k * spacing[2]
                coordinates.append((x,y,z))
    dt_coordinates = pd.DataFrame(coordinates, columns=['x','y','z'])
    
    # result coord merge
    return pd.merge(dt_coordinates, dataset, left_index=True, right_index=True)

In [3]:
path_saved = '/home/hero/Git/ai_modulus/coating_modulus/outputs/'
doe_name = 'twophase_coat_v7_1'

In [4]:
dirnames = os.listdir(path_saved+doe_name)
# 'windows'를 포함하는 이름만 선택
dir_windows = [name for name in dirnames if 'window' in name.lower()]
dir_windows.sort()

In [5]:
dir_windows

['window_0000',
 'window_0001',
 'window_0002',
 'window_0003',
 'window_0004',
 'window_0005',
 'window_0006',
 'window_0007',
 'window_0008',
 'window_0009',
 'window_0010',
 'window_0011',
 'window_0012',
 'window_0013',
 'window_0014']

In [6]:
file_path = path_saved + doe_name + '/' + dir_windows[13] + '/inferencers/window_000t.vti'
vti_data = vti_reader(file_path)

In [7]:
vti_slurry = vti_data[vti_data['alpha']==0.0].dropna()

left_max_y = vti_slurry[vti_slurry['x']<0.0]['y'].max()

pnt_left_up = vti_slurry[(vti_slurry['x']<0.0) & (vti_slurry['y']==left_max_y)]['x'].min() * L_ref
pnt_left_dn = vti_slurry[vti_slurry['y']==0.0]['x'].min() * L_ref
pnt_right_up = vti_slurry[vti_slurry['y'] == vti_slurry['y'].max()]['x'].iloc[0] * L_ref
pnt_right_dn = vti_slurry[vti_slurry['y']==0.0]['x'].min() * L_ref


In [8]:
pnt_left_up, pnt_left_dn, pnt_right_up, pnt_right_dn

(-0.00022834645669291342,
 -0.00020078740157480313,
 1.9685039370078705e-05,
 -0.00020078740157480313)

In [12]:
vti_slurry[vti_slurry['y'] == vti_slurry['y'].max()]['x']*L_ref

16293    0.000020
16294    0.000047
16295    0.000075
16296    0.000102
16297    0.000130
16298    0.000157
16299    0.000185
16300    0.000213
16301    0.000240
16302    0.000268
16303    0.000295
16304    0.000323
16305    0.000350
16306    0.000378
16307    0.000406
16308    0.000433
16309    0.000461
16310    0.000488
Name: x, dtype: float64